Load dataframes with acf and calculate tau

In [1]:
import pandas as pd
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

In [2]:
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms\\'

### Functions

In [3]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    acf_2d = acf_df_[acf_cols_].values
    acf_2d = acf_2d.astype(np.float64) # RuntimeWarning: overflow encountered in matmul sttc start_idx 2
    n_units = acf_2d.shape[0]
    print('Calculating taus for {}'.format(acf_2d.shape))
    
    fit_popt_a_l, fit_popt_b_l, fit_popt_c_l = [],[],[]
    fit_tau_l = []
    fit_r_squared_l = []
    fit_log_message_l = []
    
    for i in range(n_units):
        fit_popt, fit_pcov, tau, fit_r_squared, log_message = fit_single_exp(acf_2d[i,:], start_idx_, func_single_exp_monkey)
        if  type(fit_popt) == np.ndarray:
            fit_popt_a_l.append(fit_popt[0])
            fit_popt_b_l.append(fit_popt[1])
            fit_popt_c_l.append(fit_popt[2])
        else:
            fit_popt_a_l.append(np.nan)
            fit_popt_b_l.append(np.nan)
            fit_popt_c_l.append(np.nan)
        fit_tau_l.append(tau)
        fit_r_squared_l.append(fit_r_squared)
        fit_log_message_l.append(log_message)
    
    data_df = np.vstack((fit_popt_a_l, fit_popt_b_l, fit_popt_c_l, fit_tau_l, fit_r_squared_l, fit_log_message_l)).T
    tau_df = pd.DataFrame(data_df, columns=['fit_a', 'fit_b', 'fit_c','tau', 'r_squared', 'log_message'])
    tau_df.insert(0, 'unit_id', acf_df_['unit_id'].values)

    for col in ['fit_a', 'fit_b', 'fit_c','tau', 'r_squared']:
        tau_df[col] = tau_df[col].astype(float) 
    
    return tau_df

In [4]:
# different way of calculating tau per area 
def calc_tau_area(acf_df_, acf_cols_, start_idx_=1):
    acf_2d = acf_df_[acf_cols_].values
    print(acf_2d.shape)
    
    t = np.linspace(start_idx_, acf_2d.shape[1]-start_idx_, acf_2d.shape[1]-start_idx_).astype(int)
    # print(t)
    
    # make 1d for curve_fit
    acf_1d = np.hstack(acf_2d[:,1:])
    print(acf_1d.shape)
    t_1d = np.tile(t, reps=acf_2d.shape[0])
    print(t_1d.shape)
    
    popt, pcov = curve_fit(func_single_exp, t_1d, acf_1d, maxfev=1000000000) # I used 5000, now it is like in Siegle
    tau = 1 / popt[1]
    
    # fit r-squared
    y_pred = func_single_exp(t_1d, *popt)
    fit_r_squared = r2_score(acf_1d, y_pred)

    return tau, popt, fit_r_squared

### Calculate tau

#### One tau per area

2 methods of doing that:
* average ACF over units and fit
* fit using all ACFs from all neurons

In [5]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


##### average ACF over units and fit

In [6]:
area = 'pfdl' 

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d_mean = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_pearsonr_trial_avg_fit_popt, _, pfdl_pearsonr_trial_avg_tau, pfdl_pearsonr_trial_avg_fit_r_squared, _ = fit_single_exp(pearsonr_trial_avg_2d_mean, 
                                                                            start_idx_=2, exp_fun_=func_single_exp_monkey)
pfdl_pearsonr_trial_avg_tau_ms = pfdl_pearsonr_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_pearsonr_trial_avg_tau_ms, 
                                                             pfdl_pearsonr_trial_avg_fit_popt, 
                                                             pfdl_pearsonr_trial_avg_fit_r_squared))

sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_avg_2d_mean = np.nanmean(sttc_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_sttc_trial_avg_fit_popt, _, pfdl_sttc_trial_avg_tau, pfdl_sttc_trial_avg_fit_r_squared, _ = fit_single_exp(sttc_trial_avg_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfdl_sttc_trial_avg_tau_ms = pfdl_sttc_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_sttc_trial_avg_tau_ms, 
                                                             pfdl_sttc_trial_avg_fit_popt, 
                                                             pfdl_sttc_trial_avg_fit_r_squared))

sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_concat_2d_mean = np.nanmean(sttc_trial_concat_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_sttc_trial_concat_fit_popt, _, pfdl_sttc_trial_concat_tau, pfdl_sttc_trial_concat_fit_r_squared, _ = fit_single_exp(sttc_trial_concat_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfdl_sttc_trial_concat_tau_ms = pfdl_sttc_trial_concat_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_sttc_trial_concat_tau_ms, 
                                                             pfdl_sttc_trial_concat_fit_popt, 
                                                             pfdl_sttc_trial_concat_fit_r_squared))

tau 240.75047875443357, popt [0.21563292 0.20768391 0.1519869 ], fit_r_squared 0.9954832781393308
tau 243.25030236826842, popt [0.17114559 0.20554959 0.56114312], fit_r_squared 0.9962669955022356
tau 179.2073997377251, popt [0.31666675 0.27900634 0.19925986], fit_r_squared 0.9760184400425521


In [7]:
area = 'pfp' 

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d_mean = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_pearsonr_trial_avg_fit_popt, _, pfp_pearsonr_trial_avg_tau, pfp_pearsonr_trial_avg_fit_r_squared, _ = fit_single_exp(pearsonr_trial_avg_2d_mean, 
                                                                            start_idx_=2, exp_fun_=func_single_exp_monkey)
pfp_pearsonr_trial_avg_tau_ms = pfp_pearsonr_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_pearsonr_trial_avg_tau_ms, 
                                                             pfp_pearsonr_trial_avg_fit_popt, 
                                                             pfp_pearsonr_trial_avg_fit_r_squared))

sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_avg_2d_mean = np.nanmean(sttc_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_sttc_trial_avg_fit_popt, _, pfp_sttc_trial_avg_tau, pfp_sttc_trial_avg_fit_r_squared, _ = fit_single_exp(sttc_trial_avg_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfp_sttc_trial_avg_tau_ms = pfp_sttc_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_sttc_trial_avg_tau_ms, 
                                                             pfp_sttc_trial_avg_fit_popt, 
                                                             pfp_sttc_trial_avg_fit_r_squared))

sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_concat_2d_mean = np.nanmean(sttc_trial_concat_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_sttc_trial_concat_fit_popt, _, pfp_sttc_trial_concat_tau, pfp_sttc_trial_concat_fit_r_squared, _ = fit_single_exp(sttc_trial_concat_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfp_sttc_trial_concat_tau_ms = pfp_sttc_trial_concat_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_sttc_trial_concat_tau_ms, 
                                                             pfp_sttc_trial_concat_fit_popt, 
                                                             pfp_sttc_trial_concat_fit_r_squared))

tau 223.54058813497463, popt [0.17090143 0.22367303 0.21222378], fit_r_squared 0.992286742303797
tau 222.75020201159091, popt [0.13212734 0.22446669 0.6686685 ], fit_r_squared 0.9893918617881063
tau 198.14650064517286, popt [0.22805424 0.25233855 0.25476348], fit_r_squared 0.9717247849009442


#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [ ]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

In [ ]:
area = 'pfdl' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

In [ ]:
area = 'pfp' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

In [4]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [40]:
area = 'pfdl' 

# pearsonr 
# pearsonr_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(pearsonr_per_trial_df)}, dropping before tau calc ...')
# pearsonr_per_trial_df.dropna(inplace=True)
# pearsonr_per_trial_df.reset_index(drop=True, inplace=True)
# tau_pearsonr_per_trial_df = calc_tau_per_unit(pearsonr_per_trial_df, acf_cols[:-1], start_idx_=1)
# tau_pearsonr_per_trial_df['tau_ms'] = tau_pearsonr_per_trial_df['tau'] * bin_size
# tau_pearsonr_per_trial_df.insert(1, 'trial_id', pearsonr_per_trial_df['trial_id'])
# tau_pearsonr_per_trial_df.insert(2, 'condition_id', pearsonr_per_trial_df['condition_id'])
# tau_pearsonr_per_trial_df.insert(3, 'spike_count', pearsonr_per_trial_df['spike_count'])
# tau_pearsonr_per_trial_df.insert(4, 'fr_hz', pearsonr_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_pearsonr_per_trial_df)}')
# tau_pearsonr_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # acf formula
# pcf_proper_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pcf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(pcf_proper_per_trial_df)}, dropping before tau calc ...')
# pcf_proper_per_trial_df.dropna(inplace=True)
# pcf_proper_per_trial_df.reset_index(drop=True, inplace=True)
# tau_acf_proper_per_trial_df = calc_tau_per_unit(pcf_proper_per_trial_df, acf_cols, start_idx_=1)
# tau_acf_proper_per_trial_df['tau_ms'] = tau_acf_proper_per_trial_df['tau'] * bin_size
# tau_acf_proper_per_trial_df.insert(1, 'trial_id', pcf_proper_per_trial_df['trial_id'])
# tau_acf_proper_per_trial_df.insert(2, 'condition_id', pcf_proper_per_trial_df['condition_id'])
# tau_acf_proper_per_trial_df.insert(3, 'spike_count', pcf_proper_per_trial_df['spike_count'])
# tau_acf_proper_per_trial_df.insert(4, 'fr_hz', pcf_proper_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_acf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_acf_proper_per_trial_df)}')
# tau_acf_proper_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# isttc
sttc_per_trial_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
print(f'N rows with NaN {sttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(sttc_per_trial_df)}, dropping before tau calc ...')
sttc_per_trial_df.dropna(inplace=True)
sttc_per_trial_df.reset_index(drop=True, inplace=True)
tau_isttc_per_trial_df = calc_tau_per_unit(sttc_per_trial_df, acf_cols, start_idx_=1)
tau_isttc_per_trial_df['tau_ms'] = tau_isttc_per_trial_df['tau'] * bin_size
tau_isttc_per_trial_df.insert(1, 'trial_id', sttc_per_trial_df['trial_id'])
tau_isttc_per_trial_df.insert(2, 'condition_id', sttc_per_trial_df['condition_id'])
tau_isttc_per_trial_df.insert(3, 'spike_count', sttc_per_trial_df['spike_count'])
tau_isttc_per_trial_df.insert(4, 'fr_hz', sttc_per_trial_df['fr_hz'])
print(f'N rows with NaN {tau_isttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_isttc_per_trial_df)}')
tau_isttc_per_trial_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

N rows with NaN 0 out of 23043, dropping before tau calc ...
Calculating taus for (23043, 20)


KeyboardInterrupt: 

In [39]:
tau_acf_proper_per_trial_df

,unit_id,trial_id,condition_id,spike_count,fr_hz,fit_a,fit_b,fit_c,tau,r_squared,log_message,tau_ms
0,0,0,0,12,12.0,-0.793724,2.952993,3.025541e-02,0.338640,0.007156,ok,1.693198e+01
1,0,1,0,11,11.0,-0.136542,0.367918,7.453870e-02,2.717998,0.027611,ok,1.358999e+02
2,0,2,0,16,16.0,-165.041750,0.000011,-9.997278e-01,88704.372968,0.004029,ok,4.435219e+06
3,0,3,0,36,36.0,-207.179875,8.363594,1.147326e-04,0.119566,0.013294,ok,5.978291e+00
4,0,4,0,8,8.0,0.741975,0.497282,-1.171555e-01,2.010931,0.550560,ok,1.005465e+02
...,...,...,...,...,...,...,...,...,...,...,...,...
22415,542,95,7,9,9.0,1.994727,0.921154,-4.799761e-02,1.085595,0.669802,ok,5.427975e+01
22416,542,97,7,4,4.0,73690.489600,12.542316,-5.451045e-07,0.079730,0.262755,ok,3.986505e+00
22417,542,102,7,3,3.0,-22313.898989,12.516061,9.862373e-07,0.079897,0.028499,ok,3.994867e+00
22418,542,106,7,3,3.0,-145.345229,-0.000015,-9.999708e-01,-65880.477604,0.018376,ok,-3.294024e+06


In [28]:
pearsonr_per_trial_df

,unit_id,trial_id,condition_id,spike_count,fr_hz,acf_0,acf_1,acf_2,acf_3,acf_4,...,acf_9,acf_10,acf_11,acf_12,acf_13,acf_14,acf_15,acf_16,acf_17,acf_18
2,0,2,0,16,16.0,1.0,0.316000,0.280000,-0.128928,-0.304759,...,0.000000,-0.046241,0.022942,4.173650e-01,-4.961389e-01,3.162278e-01,-0.327327,-0.301511,0.866025,-1.0
3,0,3,0,36,36.0,1.0,-0.087161,-0.022108,-0.015973,0.044655,...,-0.125357,0.264135,-0.580948,4.082483e-01,-3.311331e-01,1.740777e-01,0.080064,-0.688247,0.500000,-1.0
5,0,5,0,32,32.0,1.0,0.139122,0.643883,-0.016832,0.332019,...,-0.583008,-0.357548,-0.157243,-2.844401e-01,-1.414214e-01,8.574929e-02,-0.060193,0.333333,-0.944911,1.0
6,0,6,0,29,29.0,1.0,0.172775,-0.107736,-0.085450,-0.186543,...,0.174667,0.487950,0.047246,2.775558e-17,2.850439e-01,-1.893885e-01,0.000000,0.000000,0.944911,-1.0
11,0,11,1,35,35.0,1.0,0.022059,-0.175899,-0.348375,-0.040522,...,0.067082,0.447914,0.196116,2.182179e-01,-1.414214e-01,-6.859943e-01,0.872872,0.000000,0.000000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22354,541,88,6,22,22.0,1.0,0.534582,0.312410,0.374793,0.112306,...,-0.233854,0.043261,-0.452267,9.990015e-02,7.156264e-01,-2.236068e-01,-0.050252,0.816497,-0.500000,-1.0
22357,541,91,7,18,18.0,1.0,0.321429,-0.036603,0.203626,0.175655,...,-0.104713,-0.450443,0.209165,7.533708e-01,2.105380e-01,8.304548e-02,-0.375000,-0.333333,1.000000,-1.0
22360,541,94,7,16,16.0,1.0,0.300613,-0.090909,-0.332090,-0.338912,...,-0.225668,0.071392,-0.153897,-2.344675e-01,-2.401922e-01,6.938894e-18,0.918559,-0.333333,-0.500000,-1.0
22362,541,96,7,12,12.0,1.0,0.240759,-0.208054,0.066184,0.016464,...,-0.023187,-0.327327,-0.100000,-5.374838e-01,-4.183300e-01,5.477226e-01,0.801784,-0.577350,0.500000,1.0


### todo (metrics)

In [ ]:
n_rows_with_nans = pearsonr_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(pearsonr_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_pearsonr_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_pearsonr_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))

In [ ]:
n_rows_with_nans = sttc_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(sttc_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_sttc_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_sttc_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))